In [1]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import string
import nltk
from nltk.stem import PorterStemmer
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
import spacy
from tqdm import tqdm

In [2]:
def elnum(cad): #eliminamos numeros puntuación, etc. regresa cadena
	SN = ''.join([i for i in cad if not i.isdigit()])
	clean_text = ''.join([char for char in SN if char not in string.punctuation])
	tlimp=clean_text.lower()
	dd=tlimp.split(" ")
	term=[]
	for i in dd:
		if i.__contains__(' ')==False and i.__contains__('\n')==False:
			term.append(i)
	ttt=' '.join(term)
	return ttt

In [3]:
def limpieza(cad): #regresa un docuemnto(sin stop-words)
	nlp = spacy.load("en_core_web_sm")
	stop_words = spacy.lang.en.stop_words.STOP_WORDS
	doc = nlp(cad)
	filtered_tokens = [token for token in doc if not token.is_stop and token.text !=' ']
	return ' '.join([token.text for token in filtered_tokens])


In [4]:
def lemma(cad):

	nlp = spacy.load("en_core_web_sm")
	doc = nlp(cad)
	lemma_tokens = ' '.join([token.lemma_ for token in doc])
	return lemma_tokens

In [5]:
def stemm(cad):
	ps = PorterStemmer()
	arr=[]
	example_words=cad.split(" ")
	for word in example_words:
		arr.append(ps.stem(word))
	final=" ".join(arr)
	return final

In [ ]:
path="austinHousingData.csv"
data=pd.read_csv(path)
corpus=list(data['description'])
print(len(corpus))
y=list(data['latestPrice'])
document=[]
ttd=len(corpus)
for i in tqdm(range(0, ttd), desc ="Normalizando texto"):
	try:
		t1=elnum(corpus[i])
		t2=limpieza(t1)
		#lterm=lemma(t2)
		ste=stemm(t2)
		document.append(ste)
	except:
		print('el documento no se pudo pre-procesar', i)

15171


Normalizando texto:   1%|                 | 103/15171 [00:54<2:07:36,  1.97it/s]

In [ ]:
vectorizer = TfidfVectorizer() #CountVectorizer()
X = vectorizer.fit_transform(document)
print(X.shape)


NameError: name 'CountVectorizer' is not defined

In [ ]:
n=300
nom=[]
print(len(vectorizer.get_feature_names_out()))
svd = TruncatedSVD(n_components=n, n_iter=10, random_state=42)
svd.fit(X)
print(svd)
XB=svd.transform(X)
print(svd.explained_variance_ratio_.sum())

NameError: name 'vectorizer' is not defined

In [ ]:
# Obtener los nombres de las palabras como columnas
column_names = vectorizer.get_feature_names_out()

# Crear el DataFrame
dfsal = pd.DataFrame(X.toarray(), columns=column_names)
dfsal.to_csv('out.csv', index=False)